<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/6%20-%20Generatori%20e%20Images%20Augmentation/generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generatori di Immagini
Spesso capita di dover addestrare una rete neurale su una mole di dati talmente grande da non riuscire a risiedere sulla memoria del nostro PC. A questo scopo tf.keras ci mette a disposizione i generatori, uno strumento che permette di caricare le immagini in batch durante la fase di addestramento.<br>
In questo notebook vedremo come migliorare il nostro modello per riconoscere casi di malaria, sfruttando un generatore per eseguire l'addestramento su tutte le immagini

## Scarichiamo il Dataset
Questa volta, piuttosto che usare tensorflow datasets, scarichiamo la raccolta di immagini direttamente dal sito ufficiale.

In [1]:
!wget ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip
!unzip -qq cell_images.zip

--2020-07-03 14:31:49--  ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip
           => ‘cell_images.zip’
Resolving lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)... 130.14.55.35, 2607:f220:41e:7055::35
Connecting to lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)|130.14.55.35|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /Open-Access-Datasets/Malaria ... done.
==> SIZE cell_images.zip ... 353452851
==> PASV ... done.    ==> RETR cell_images.zip ... done.
Length: 353452851 (337M) (unauthoritative)

cell_images.zip     100%[===================>] 337.08M  3.22MB/s    in 36s     

2020-07-03 14:32:29 (9.27 MB/s) - ‘cell_images.zip’ saved [353452851]



Le immagini sono raccolte dentro due sotto-cartelle:
* **Parasitized**: contiene le immagini di cellule di pazienti infetti.
* **Uninfected**: contiene le immagini di cellule di pazienti sani.

## Importiamo i Moduli

In [3]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Definiamo le Costanti

In [16]:
DATASET_DIR = "cell_images/"
BATCH_SIZE = 64

TOT_IMAGES = 27558 
IMG_SIZE = (228, 228)
VALIDATION_SPLIT = 0.1

## Creiamo il Generatore

Possiamo creare il generatore utilizzando la classe *ImageDataGenerator* di tf.keras, al suo interno possiamo definire la dimensione del set di validazione.

In [4]:
datagen_train = ImageDataGenerator(validation_split=VALIDATION_SPLIT rescale=1./255,)

tramite il parametro *rescale* abbiamo definito come normalizzare le immagini, cioè dividendo per 255.<br>
Ora per creare i generatori di immagini, utilizziamo il metodo *flow_from_directory*, specificando:
 - il path al set di immagini
 - *target_size*: la dimensione da utilizzare per le immagini
 - *batch_size*: la dimensione di ogni batch di immagini
 - *class_mode*: la tipologia di classificazione, multiclasse (categorical) o binaria (binary).
 - *subset*: qui possiamo specificare se si tratta del set di training o di validazione.

In [8]:
train_generator = datagen_train.flow_from_directory(
        DATASET_DIR,
        target_size=IMG_SIZE, 
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='training')

valid_generator = datagen_train.flow_from_directory(
        DATASET_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='validation')

print(train_generator.class_indices)

Found 24804 images belonging to 2 classes.
Found 2754 images belonging to 2 classes.
{'Parasitized': 0, 'Uninfected': 1}


## Addestriamo la Rete Neurale Convoluzionale sul Generatore
Definiamo l'architettura della rete, trattandosi di un problema abbastanza complesso utilizzeremo diversi strati, insieme a dropout e regolarizzazione L2 per cercare di limitare l'overfiting.

In [24]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 228, 228, 32)      416       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 76, 76, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 76, 76, 32)        4128      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 25, 25, 64)        8256      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
dense_13 (Dense)             (None, 8, 8, 128)        

Per avviare l'addestramento ci basterà passare al metodo *fit* il generatore di addestramento e l'eventuale generatore di validazione. Dobbiamo solo stare attenti a specificare il numero di steps per ogni epoca del Gradient Descent, perché quando utilizziamo i generatori Tensorflow non è in grado di ottenere questo valore automaticamente, solitamente basta utilizzare: NUMERO_DI_IMMAGINI_NEL_SET/DIMENSIONE_DEL_BATCH.

In [25]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                              min_delta=0.01,
                              patience=3,
                              restore_best_weights=True)

model.fit(
        train_generator,
        steps_per_epoch=int(TOT_IMAGES*(1.-VALIDATION_SPLIT) // BATCH_SIZE),
        validation_data=valid_generator, 
        validation_steps=int(TOT_IMAGES*VALIDATION_SPLIT // BATCH_SIZE),
        epochs=5)

Epoch 1/5
387/387 [==============================] - 58s 149ms/step - loss: 0.3684 - accuracy: 0.8241 - val_loss: 0.2291 - val_accuracy: 0.9182
Epoch 2/5
387/387 [==============================] - 57s 148ms/step - loss: 0.1595 - accuracy: 0.9531 - val_loss: 0.1967 - val_accuracy: 0.9266
Epoch 3/5
387/387 [==============================] - 57s 147ms/step - loss: 0.1412 - accuracy: 0.9583 - val_loss: 0.2155 - val_accuracy: 0.9288
Epoch 4/5
387/387 [==============================] - 57s 147ms/step - loss: 0.1342 - accuracy: 0.9594 - val_loss: 0.2069 - val_accuracy: 0.9306
Epoch 5/5
387/387 [==============================] - 57s 148ms/step - loss: 0.1255 - accuracy: 0.9612 - val_loss: 0.1972 - val_accuracy: 0.9339


# Testiamo la Rete sul Generatore
Per testare sul generatore la Rete Neurale che abbiamo addestrato, ci basta passare il generatore al metodo *evaluate*.

In [26]:
metrics_train = model.evaluate(train_generator)
metrics_valid = model.evaluate(valid_generator)

print("Train Accuracy = %.4f - Train Loss = %.4f" % (metrics_train[1], metrics_train[0]))
print("Validation Accuracy = %.4f - Validation Loss = %.4f" % (metrics_valid[1], metrics_valid[0]))

44/44 [==============================] - 5s 116ms/step - loss: 0.1971 - accuracy: 0.9339
Train Accuracy = 0.9645 - Train Loss = 0.1067
Validation Accuracy = 0.9339 - Validation Loss = 0.1971
